## SNOMED CT(v.1.2.2)

In [44]:
import pandas as pd

df = pd.read_csv('/Users/rose/omop-mapper/data/snomed-ct-entity-challenge/train_annotations.csv')
df

,annotation_id,note_id,start,end,span,concept_id,annotation_type
0,1,10043750-DS-6,180.0,198.0,No Known Allergies,609328004,test
1,2,10043750-DS-6,201.0,223.0,Adverse Drug Reactions,419511003,test
2,3,10043750-DS-6,263.0,275.0,colon cancer,363406005,test
3,4,10043750-DS-6,321.0,336.0,right colectomy,359571009,test
4,5,10043750-DS-6,393.0,399.0,clinic,737492002,test
...,...,...,...,...,...,...,...
75486,265076,11532659-DS-11,10592.0,10606.0,hemoperitoneum,45626005,proposed_ACCEPTED
75487,265078,10860165-DS-24,6894.0,6910.0,hemopneumothorax,16632002,proposed_ACCEPTED
75488,265229,13227028-DS-12,3373.0,3389.0,intraparenchymal,449020009,proposed_ACCEPTED
75489,265266,11714071-DS-56,5982.0,5985.0,jaw,661005,proposed_ACCEPTED


In [45]:
df = df.rename(columns={'concept_id': 'entity_code'})
df

,annotation_id,note_id,start,end,span,entity_code,annotation_type
0,1,10043750-DS-6,180.0,198.0,No Known Allergies,609328004,test
1,2,10043750-DS-6,201.0,223.0,Adverse Drug Reactions,419511003,test
2,3,10043750-DS-6,263.0,275.0,colon cancer,363406005,test
3,4,10043750-DS-6,321.0,336.0,right colectomy,359571009,test
4,5,10043750-DS-6,393.0,399.0,clinic,737492002,test
...,...,...,...,...,...,...,...
75486,265076,11532659-DS-11,10592.0,10606.0,hemoperitoneum,45626005,proposed_ACCEPTED
75487,265078,10860165-DS-24,6894.0,6910.0,hemopneumothorax,16632002,proposed_ACCEPTED
75488,265229,13227028-DS-12,3373.0,3389.0,intraparenchymal,449020009,proposed_ACCEPTED
75489,265266,11714071-DS-56,5982.0,5985.0,jaw,661005,proposed_ACCEPTED


In [46]:
import os

from elasticsearch import Elasticsearch

ES_SERVER_HOST = os.getenv("ES_SERVER_HOST", "3.35.110.161")
ES_SERVER_PORT = int(os.getenv("ES_SERVER_PORT", "9200"))
ES_SERVER_USERNAME = os.getenv("ES_SERVER_USERNAME", "elastic")
ES_SERVER_PASSWORD = os.getenv("ES_SERVER_PASSWORD", "snomed")


es_client = Elasticsearch(
    f"http://{ES_SERVER_HOST}:{ES_SERVER_PORT}",
    basic_auth=(ES_SERVER_USERNAME, ES_SERVER_PASSWORD),
    request_timeout=60,
)

In [47]:
import warnings

import pandas as pd
from tqdm import tqdm


def map_snomed_to_omop_es(input_df: pd.DataFrame) -> pd.DataFrame:
    """
    DataFrame의 'entity_code'를 사용하여 Elasticsearch 'concept-small' 인덱스를 조회하고,
    OMOP concept_id, concept_name, domain_id를 병합합니다.

    Args:
        input_df (pd.DataFrame): 'note_id', 'note_text', 'entity_name', 'entity_code' 포함

    Returns:
        pd.DataFrame: 'concept_id', 'concept_name', 'domain_id'가 추가된 DataFrame
    """

    # 2. 고유한 entity_code 추출 (ES 조회의 효율성을 위해)
    # 'entity_code'를 문자열로 변환 (ES term 쿼리 및 merge를 위해)
    try:
        input_df["entity_code"] = input_df["entity_code"].astype(str)
    except Exception as e:
        print(f"[오류] 'entity_code'를 문자열로 변환 중 오류: {e}")
        # NaN이나 비정상 값이 있는지 확인 필요
        input_df = input_df.dropna(subset=["entity_code"])
        input_df["entity_code"] = input_df["entity_code"].astype(int).astype(str)

    unique_codes = input_df["entity_code"].unique()
    print(f"\n--- 2. 고유 entity_code 추출 완료 ---")
    print(f"원본 {len(input_df)}건 -> 고유 코드 {len(unique_codes)}건")

    # 3. ES 조회 및 매핑 테이블 생성
    print(f"\n--- 3. ES 인덱스 'concept-small' 조회 중 ---")
    snomed_to_omop_map = {}

    for code in tqdm(unique_codes, desc="Mapping SNOMED codes"):
        # ES 쿼리 정의
        query_body = {
            "query": {
                "bool": {
                    "filter": [
                        {"term": {"vocabulary_id": "SNOMED"}},
                        {"term": {"concept_code": code}},
                        # 'concept_code'가 'keyword' 타입으로 매핑되어 있어야 함
                    ]
                }
            },
            "_source": [
                "concept_id",
                "concept_name",
                "domain_id",
                "standard_concept",
            ],  # 표준 여부도 확인
            "size": 1,  # concept_code + vocabulary_id는 고유해야 함
        }

        try:
            response = es_client.search(index="concept-small", body=query_body)
            hits = response["hits"]["hits"]

            if hits:
                # 직접 매핑된 컨셉 (Standard 또는 Non-Standard일 수 있음)
                source_concept = hits[0]["_source"]

                # 여기서는 사용자의 요청(직접 조회)을 그대로 따릅니다.
                snomed_to_omop_map[code] = {
                    "concept_id": source_concept.get("concept_id"),
                    "concept_name": source_concept.get("concept_name"),
                    "domain_id": source_concept.get("domain_id"),
                }
            else:
                # ES에서 코드를 찾지 못한 경우
                snomed_to_omop_map[code] = {
                    "concept_id": None,
                    "concept_name": None,
                    "domain_id": None,
                }
        except Exception as e:
            print(f"ES 조회 중 오류 (Code: {code}): {e}")
            snomed_to_omop_map[code] = {
                "concept_id": None,
                "concept_name": None,
                "domain_id": None,
            }

    print("ES 조회 및 매핑 테이블 생성 완료.")

    # 4. 매핑 딕셔너리를 DataFrame으로 변환
    print("\n--- 4. 매핑 테이블 -> DataFrame 변환 ---")
    omop_df = pd.DataFrame.from_dict(snomed_to_omop_map, orient="index")
    omop_df = omop_df.reset_index().rename(columns={"index": "entity_code"})

    # entity_code를 merge를 위해 문자열 타입으로 유지
    omop_df["entity_code"] = omop_df["entity_code"].astype(str)

    # 5. 원본 DataFrame과 매핑된 OMOP 정보 병합
    print("\n--- 5. 원본 DataFrame과 병합 (Left Join) ---")
    final_df = pd.merge(input_df, omop_df, on="entity_code", how="left")

    print("병합 완료! 최종 데이터프레임 생성.")
    return final_df


# --- 스크립트 실행 ---
if __name__ == "__main__":
    if "df" in locals() or "df" in globals():
        print("--- 'df' 변수 확인됨 ---")
        final_mapped_df = map_snomed_to_omop_es(df)

        print("\n\n--- 최종 매핑 결과 (상위 10개) ---")
        print(final_mapped_df.head(10))

        print("\n\n--- 매핑 실패 (None) 건수 확인 ---")
        print(
            final_mapped_df["concept_id"].isna().sum(),
            "건의 매핑 실패 (ES에서 찾을 수 없음)",
        )

    else:
        print("[주의] 'mapping_test_df' 변수가 존재하지 않습니다.")
        print("이 스크립트를 단독으로 실행한 경우, 테스트용 더미 데이터를 사용하거나")
        print("이전 단계의 'create_mapping_test_set()' 함수를 먼저 실행해야 합니다.")

--- 'df' 변수 확인됨 ---

--- 2. 고유 entity_code 추출 완료 ---
원본 75491건 -> 고유 코드 6595건

--- 3. ES 인덱스 'concept-small' 조회 중 ---


Mapping SNOMED codes:   0%|          | 0/6595 [00:00<?, ?it/s]

Mapping SNOMED codes: 100%|██████████| 6595/6595 [06:23<00:00, 17.21it/s]  

ES 조회 및 매핑 테이블 생성 완료.

--- 4. 매핑 테이블 -> DataFrame 변환 ---

--- 5. 원본 DataFrame과 병합 (Left Join) ---
병합 완료! 최종 데이터프레임 생성.


--- 최종 매핑 결과 (상위 10개) ---
   annotation_id        note_id  start    end                    span  \
0              1  10043750-DS-6  180.0  198.0      No Known Allergies   
1              2  10043750-DS-6  201.0  223.0  Adverse Drug Reactions   
2              3  10043750-DS-6  263.0  275.0            colon cancer   
3              4  10043750-DS-6  321.0  336.0         right colectomy   
4              5  10043750-DS-6  393.0  399.0                  clinic   
5              6  10043750-DS-6  426.0  439.0            colon cancer   
6              7  10043750-DS-6  472.0  483.0             colonoscopy   
7              8  10043750-DS-6  504.0  516.0            cecal cancer   
8              9  10043750-DS-6  526.0  540.0          investigations   
9             10  10043750-DS-6  558.0  576.0      metastatic disease   

  entity_code annotation_type concept_id  \
0   6

In [48]:
final_mapped_df = final_mapped_df.drop(columns=["annotation_id", "start", "end", "entity_code", "annotation_type"])
final_mapped_df

,note_id,span,concept_id,concept_name,domain_id
0,10043750-DS-6,No Known Allergies,43530807,allergic disposition,Observation
1,10043750-DS-6,Adverse Drug Reactions,4305025,propensity to adverse reactions to drug,Observation
2,10043750-DS-6,colon cancer,4180790,malignant tumor of colon,Condition
3,10043750-DS-6,right colectomy,4233412,right colectomy,Procedure
4,10043750-DS-6,clinic,42537845,outpatient care management,Procedure
...,...,...,...,...,...
75486,11532659-DS-11,hemoperitoneum,194690,nontraumatic hemoperitoneum,Condition
75487,10860165-DS-24,hemopneumothorax,4042409,hemopneumothorax,Condition
75488,13227028-DS-12,intraparenchymal,40492969,intraparenchymal hemorrhage of brain,Condition
75489,11714071-DS-56,jaw,4281237,jaw region structure,Spec Anatomic Site


In [49]:
final_mapped_df = final_mapped_df.rename(columns={'span': 'entity_name'})
final_mapped_df.head()

,note_id,entity_name,concept_id,concept_name,domain_id
0,10043750-DS-6,No Known Allergies,43530807,allergic disposition,Observation
1,10043750-DS-6,Adverse Drug Reactions,4305025,propensity to adverse reactions to drug,Observation
2,10043750-DS-6,colon cancer,4180790,malignant tumor of colon,Condition
3,10043750-DS-6,right colectomy,4233412,right colectomy,Procedure
4,10043750-DS-6,clinic,42537845,outpatient care management,Procedure


In [9]:
final_mapped_df['concept_id'].unique()

array(['43530807', '4305025', '4180790', ..., '602851', '4107785',
       '4046477'], dtype=object)

In [16]:
final_mapped_df[final_mapped_df['concept_id'] == '43530807']

,note_id,entity_name,concept_id,concept_name,domain_id
0,10043750-DS-6,No Known Allergies,43530807,allergic disposition,Observation
5681,11532659-DS-11,No Known Allergies,43530807,allergic disposition,Observation
6351,11576109-DS-15,No Known Allergies,43530807,allergic disposition,Observation
9623,12050253-DS-20,allergy,43530807,allergic disposition,Observation
9666,12128814-DS-15,No Known Allergies,43530807,allergic disposition,Observation
11286,12549331-DS-3,No Known Allergies,43530807,allergic disposition,Observation
12185,12626414-DS-32,No Known Allergies,43530807,allergic disposition,Observation
12804,12962702-DS-14,No Known Allergies,43530807,allergic disposition,Observation
18949,16393593-DS-5,No Known Allergies,43530807,allergic disposition,Observation
19888,16991646-DS-11,No Known Allergies,43530807,allergic disposition,Observation


In [50]:
dedup_df = final_mapped_df.drop_duplicates(subset=['concept_id', 'entity_name'])
len(dedup_df)

16531

In [51]:
dedup_df[dedup_df['concept_id'] == '43530807']

,note_id,entity_name,concept_id,concept_name,domain_id
0,10043750-DS-6,No Known Allergies,43530807,allergic disposition,Observation
9623,12050253-DS-20,allergy,43530807,allergic disposition,Observation
22214,18752997-DS-9,ALLERGY,43530807,allergic disposition,Observation


In [52]:
dedup_df = dedup_df[dedup_df['domain_id'] != 'Spec Anatomic Site']
dedup_df['domain_id'].unique()

array(['Observation', 'Condition', 'Procedure', 'Measurement'],
      dtype=object)

In [53]:
dedup_df = dedup_df.reset_index(drop=True)
dedup_df

,note_id,entity_name,concept_id,concept_name,domain_id
0,10043750-DS-6,No Known Allergies,43530807,allergic disposition,Observation
1,10043750-DS-6,Adverse Drug Reactions,4305025,propensity to adverse reactions to drug,Observation
2,10043750-DS-6,colon cancer,4180790,malignant tumor of colon,Condition
3,10043750-DS-6,right colectomy,4233412,right colectomy,Procedure
4,10043750-DS-6,clinic,42537845,outpatient care management,Procedure
...,...,...,...,...,...
13989,19926965-DS-14,skip,4107785,does skip,Observation
13990,19926965-DS-14,deep breathing exercises,4046477,deep breathing exercises,Procedure
13991,10653756-DS-22,exercise stress test,4261794,electrocardiogram with exercise test,Measurement
13992,12970259-DS-4,NaN,4208938,blood sodium measurement (procedure),Measurement


In [54]:
dedup_df.to_csv("/Users/rose/omop-mapper/data/snomed-mapping-data-2602.csv", index=False)

In [2]:
import pandas as pd

# CSV 로드
df = pd.read_csv('/Users/rose/omop-mapper/data/snomed-mapping-data-2602.csv')

# entity_name에서 줄바꿈 제거 (\n, \r, \r\n 등)
df['entity_name'] = df['entity_name'].astype(str).str.replace(r'[\r\n]+', ' ', regex=True).str.strip()

# 여러 공백을 하나로 정리
df['entity_name'] = df['entity_name'].str.replace(r'\s+', ' ', regex=True)

In [3]:
# 줄바꿈 제거 후 CSV로 저장
df.to_csv('/Users/rose/omop-mapper/data/snomed-mapping-data-2602.csv', index=False)

## SNUH(coverage90)

In [55]:
import pandas as pd

df = pd.read_csv('/Users/rose/omop-mapper/data/MapOMOP_test_data_coverage90_concept_id.csv')
df

,record_count,domain_id,source_value,source_code,concept_id,cumulative_record_count,total_record_count
0,7399,Condition,Gynecomastia,D00014430,4168447,85375132,94863350
1,7407,Condition,lung disease,75538,257907,85367733,94863350
2,7410,Condition,"Genu valgum, acquired",D00012970,80504,85360326,94863350
3,7411,Condition,Basal ganglia hemorrhage,D00009963,4112018,85352916,94863350
4,7412,Condition,"Ankylosing spondylitis, lumbar region",D00023451,4035614,85345505,94863350
...,...,...,...,...,...,...,...
2923,94305864,Measurement,HR,1021,3027018,285293665,2670570628
2924,95477521,Measurement,DBP,1013,3012888,190987801,2670570628
2925,95477521,Measurement,"difficulty, writing",1013,3012888,190987801,2670570628
2926,95510280,Measurement,"difficulty, weaning",1012,3004249,95510280,2670570628


In [58]:
df = df.drop(columns=["record_count", "cumulative_record_count", "total_record_count"])
df

,domain_id,source_value,source_code,concept_id
0,Condition,Gynecomastia,D00014430,4168447
1,Condition,lung disease,75538,257907
2,Condition,"Genu valgum, acquired",D00012970,80504
3,Condition,Basal ganglia hemorrhage,D00009963,4112018
4,Condition,"Ankylosing spondylitis, lumbar region",D00023451,4035614
...,...,...,...,...
2923,Measurement,HR,1021,3027018
2924,Measurement,DBP,1013,3012888
2925,Measurement,"difficulty, writing",1013,3012888
2926,Measurement,"difficulty, weaning",1012,3004249


In [60]:
from tqdm import tqdm

# concept_id를 문자열이 아닌 정수로 조회해야 할 수 있으므로 확인
unique_ids = df["concept_id"].unique()
print(f"고유 concept_id: {len(unique_ids)}건")

id_to_name = {}

for cid in tqdm(unique_ids, desc="Fetching concept_name"):
    query_body = {
        "query": {
            "term": {"concept_id": int(cid)}
        },
        "_source": ["concept_name"],
        "size": 1
    }
    try:
        response = es_client.search(index="concept-small", body=query_body)
        hits = response["hits"]["hits"]
        if hits:
            id_to_name[cid] = hits[0]["_source"].get("concept_name")
        else:
            id_to_name[cid] = None
    except Exception as e:
        print(f"오류 (concept_id: {cid}): {e}")
        id_to_name[cid] = None

df["concept_name"] = df["concept_id"].map(id_to_name)
print(f"매핑 완료. 실패: {df['concept_name'].isna().sum()}건")
df.head()

고유 concept_id: 2836건


Fetching concept_name: 100%|██████████| 2836/2836 [00:45<00:00, 62.09it/s]

매핑 완료. 실패: 4건


,domain_id,source_value,source_code,concept_id,concept_name
0,Condition,Gynecomastia,D00014430,4168447,gynecomastia
1,Condition,lung disease,75538,257907,disorder of lung
2,Condition,"Genu valgum, acquired",D00012970,80504,acquired genu valgum
3,Condition,Basal ganglia hemorrhage,D00009963,4112018,basal ganglia hemorrhage
4,Condition,"Ankylosing spondylitis, lumbar region",D00023451,4035614,ankylosing spondylitis with organ / system inv...


In [63]:
df = df.dropna(subset=['concept_name'])

In [65]:
df = df.drop_duplicates(subset=['source_value', 'concept_id']).reset_index(drop=True)
df

,domain_id,source_value,source_code,concept_id,concept_name
0,Condition,Gynecomastia,D00014430,4168447,gynecomastia
1,Condition,lung disease,75538,257907,disorder of lung
2,Condition,"Genu valgum, acquired",D00012970,80504,acquired genu valgum
3,Condition,Basal ganglia hemorrhage,D00009963,4112018,basal ganglia hemorrhage
4,Condition,"Ankylosing spondylitis, lumbar region",D00023451,4035614,ankylosing spondylitis with organ / system inv...
...,...,...,...,...,...
2919,Measurement,HR,1021,3027018,heart rate
2920,Measurement,DBP,1013,3012888,diastolic blood pressure
2921,Measurement,"difficulty, writing",1013,3012888,diastolic blood pressure
2922,Measurement,"difficulty, weaning",1012,3004249,systolic blood pressure


In [72]:
df = df[df['domain_id'] != 'Observation'].reset_index(drop=True)

In [73]:
len(df)

2923

In [74]:
df.to_csv("/Users/rose/omop-mapper/data/MapOMOP_test_data_coverage90_concept_id.csv", index=False)